In [26]:
import optuna

In [46]:
import xgboost as xgb

In [28]:
import pandas as pd
import numpy as np

In [29]:
df=pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [30]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [33]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [34]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [35]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())


In [36]:
x=df.drop(columns=["Serial No.","Chance of Admit"],axis=1)

In [37]:
y=df["Chance of Admit"]

In [38]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [39]:
std_sca=StandardScaler()

In [40]:
x_train=std_sca.fit_transform(x_train)

In [41]:
x_test=std_sca.transform(x_test)

In [42]:
!nvidia-smi

Sun May 28 17:32:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [51]:
def objective(trail,data=x,target=y):
  train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=30)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :3000,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
      }
  Xgb_reg_model=xgb.XGBRegressor(**param)
  Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
  pred_xgb=Xgb_reg_model.predict(test_x)
  mse=mean_squared_error(test_y,pred_xgb)
  return mse


In [52]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=9)
find_params.best_trial.params

[I 2023-05-28 17:50:09,147] A new study created in memory with name: no-name-c88e02df-106d-4374-863c-27302938f6ec


[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25754
[4]	validation_0-rmse:0.25754
[5]	validation_0-rmse:0.25754
[6]	validation_0-rmse:0.25754
[7]	validation_0-rmse:0.25754
[8]	validation_0-rmse:0.25754
[9]	validation_0-rmse:0.25754
[10]	validation_0-rmse:0.25754
[11]	validation_0-rmse:0.25754
[12]	validation_0-rmse:0.25754
[13]	validation_0-rmse:0.25754
[14]	validation_0-rmse:0.25754
[15]	validation_0-rmse:0.25754
[16]	validation_0-rmse:0.25754
[17]	validation_0-rmse:0.25754
[18]	validation_0-rmse:0.25754
[19]	validation_0-rmse:0.25754
[20]	validation_0-rmse:0.25754
[21]	validation_0-rmse:0.25754
[22]	validation_0-rmse:0.25754
[23]	validation_0-rmse:0.25754
[24]	validation_0-rmse:0.25754
[25]	validation_0-rmse:0.25754
[26]	validation_0-rmse:0.25754
[27]	validation_0-rmse:0.25754
[28]	validation_0-rmse:0.25754
[29]	validation_0-rmse:0.25754
[30]	validation_0-rmse:0.25754
[31]	validation_0-rmse:0.25754
[32]	validation_0-

<ipython-input-51-e6deb51180f6>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-51-e6deb51180f6>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[52]	validation_0-rmse:0.25754
[53]	validation_0-rmse:0.25754
[54]	validation_0-rmse:0.25754
[55]	validation_0-rmse:0.25754
[56]	validation_0-rmse:0.25754
[57]	validation_0-rmse:0.25754
[58]	validation_0-rmse:0.25754
[59]	validation_0-rmse:0.25754
[60]	validation_0-rmse:0.25754
[61]	validation_0-rmse:0.25754
[62]	validation_0-rmse:0.25754
[63]	validation_0-rmse:0.25754
[64]	validation_0-rmse:0.25754
[65]	validation_0-rmse:0.25754
[66]	validation_0-rmse:0.25754
[67]	validation_0-rmse:0.25754
[68]	validation_0-rmse:0.25754
[69]	validation_0-rmse:0.25754
[70]	validation_0-rmse:0.25754
[71]	validation_0-rmse:0.25754
[72]	validation_0-rmse:0.25754
[73]	validation_0-rmse:0.25754
[74]	validation_0-rmse:0.25754
[75]	validation_0-rmse:0.25754
[76]	validation_0-rmse:0.25754
[77]	validation_0-rmse:0.25754
[78]	validation_0-rmse:0.25754
[79]	validation_0-rmse:0.25754
[80]	validation_0-rmse:0.25754
[81]	validation_0-rmse:0.25754
[82]	validation_0-rmse:0.25754
[83]	validation_0-rmse:0.25754
[84]	val

[I 2023-05-28 17:50:19,571] Trial 0 finished with value: 0.0663272 and parameters: {'lambda': 0.005710822674596985, 'alpha': 0.14450329062034176, 'colsample_bytree': 0.6, 'subsample': 0.3, 'learning_rate': 1, 'max_depth': 8, 'random_state': 2000, 'min_child_weight': 157}. Best is trial 0 with value: 0.0663272.


...

[22]	validation_0-rmse:0.25601
[23]	validation_0-rmse:0.25594
[24]	validation_0-rmse:0.25588
[25]	validation_0-rmse:0.25581
[26]	validation_0-rmse:0.25575
[27]	validation_0-rmse:0.25568
[28]	validation_0-rmse:0.25561
[29]	validation_0-rmse:0.25555
[30]	validation_0-rmse:0.25549
[31]	validation_0-rmse:0.25542
[32]	validation_0-rmse:0.25536
[33]	validation_0-rmse:0.25529
[34]	validation_0-rmse:0.25523
[35]	validation_0-rmse:0.25516
[36]	validation_0-rmse:0.25509
[37]	validation_0-rmse:0.25503
[38]	validation_0-rmse:0.25496
[39]	validation_0-rmse:0.25490
[40]	validation_0-rmse:0.25483
[41]	validation_0-rmse:0.25476
[42]	validation_0-rmse:0.25470
[43]	validation_0-rmse:0.25463
[44]	validation_0-rmse:0.25457
[45]	validation_0-rmse:0.25450
[46]	validation_0-rmse:0.25444
[47]	validation_0-rmse:0.25438
[48]	validation_0-rmse:0.25431
[49]	validation_0-rmse:0.25424
[50]	validation_0-rmse:0.25418
[51]	validation_0-rmse:0.25411
[52]	validation_0-rmse:0.25405
[53]	validation_0-rmse:0.25398
[54]	val

<ipython-input-51-e6deb51180f6>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-51-e6deb51180f6>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25753
[4]	validation_0-rmse:0.25753
[5]	validation_0-rmse:0.25753
[6]	validation_0-rmse:0.25753
[7]	validation_0-rmse:0.25753
[8]	validation_0-rmse:0.25752
[9]	validation_0-rmse:0.25752
[10]	validation_0-rmse:0.25752
[11]	validation_0-rmse:0.25752
[12]	validation_0-rmse:0.25752
[13]	validation_0-rmse:0.25752
[14]	validation_0-rmse:0.25751
[15]	validation_0-rmse:0.25751
[16]	validation_0-rmse:0.25751
[17]	validation_0-rmse:0.25751
[18]	validation_0-rmse:0.25751
[19]	validation_0-rmse:0.25750
[20]	validation_0-rmse:0.25750
[21]	validation_0-rmse:0.25750
[22]	validation_0-rmse:0.25750
[23]	validation_0-rmse:0.25750
[24]	validation_0-rmse:0.25750
[25]	validation_0-rmse:0.25749
[26]	validation_0-rmse:0.25749
[27]	validation_0-rmse:0.25749
[28]	validation_0-rmse:0.25749
[29]	validation_0-rmse:0.25749
[30]	validation_0-rmse:0.25748
[31]	validation_0-rmse:0.25748
[32]	validation_0-

[I 2023-05-28 17:51:50,773] Trial 8 finished with value: 0.06356428960581616 and parameters: {'lambda': 0.5073498292405102, 'alpha': 5.932629365911327, 'colsample_bytree': 0.6, 'subsample': 0.9, 'learning_rate': 1e-05, 'max_depth': 12, 'random_state': 3454, 'min_child_weight': 109}. Best is trial 6 with value: 0.004525171801933387.


{'lambda': 0.6662300719496674,
 'alpha': 0.008738322932750241,
 'colsample_bytree': 0.3,
 'subsample': 0.4,
 'learning_rate': 0.02,
 'max_depth': 3,
 'random_state': 20,
 'min_child_weight': 18}

In [53]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.066327,2023-05-28 17:50:09.151007,2023-05-28 17:50:19.571083,0 days 00:00:10.420076,0.144503,0.6,0.005711,1.00000,8,157,2000,0.3,COMPLETE
1,1,0.063204,2023-05-28 17:50:19.575458,2023-05-28 17:50:30.745911,0 days 00:00:11.170453,0.000460,0.4,8.333473,0.00001,8,131,243123,0.9,COMPLETE
2,2,0.063544,2023-05-28 17:50:30.747881,2023-05-28 17:50:41.031260,0 days 00:00:10.283379,0.001091,1.0,0.165027,0.00001,9,159,20,0.7,COMPLETE
3,3,0.063712,2023-05-28 17:50:41.034013,2023-05-28 17:50:50.571340,0 days 00:00:09.537327,3.696750,0.2,0.001148,0.00001,6,179,3454,0.7,COMPLETE
4,4,0.005521,2023-05-28 17:50:50.574474,2023-05-28 17:51:06.883092,0 days 00:00:16.308618,0.000425,0.9,0.057972,0.00800,6,2,20,0.4,COMPLETE
5,5,0.021000,2023-05-28 17:51:06.886490,2023-05-28 17:51:16.111860,0 days 00:00:09.225370,0.001441,0.2,0.004777,0.00800,6,65,30,0.2,COMPLETE
6,6,0.004525,2023-05-28 17:51:16.114843,2023-05-28 17:51:28.013126,0 days 00:00:11.898283,0.008738,0.3,0.666230,0.02000,3,18,20,0.4,COMPLETE
7,7,0.018426,2023-05-28 17:51:28.015796,2023-05-28 17:51:39.521619,0 days 00:00:11.505823,0.000948,0.6,0.931639,0.00030,6,90,20,0.6,COMPLETE
8,8,0.063564,2023-05-28 17:51:39.530485,2023-05-28 17:51:50.772962,0 days 00:00:11.242477,5.932629,0.6,0.507350,0.00001,12,109,3454,0.9,COMPLETE


In [54]:
optuna.visualization.plot_optimization_history(find_params)

In [55]:
optuna.visualization.plot_slice(find_params)

In [56]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])

In [57]:
best_params={'lambda': 0.6662300719496674,
 'alpha': 0.008738322932750241,
 'colsample_bytree': 0.3,
 'subsample': 0.4,
 'learning_rate': 0.02,
 'max_depth': 3,
 'random_state': 20,
 'min_child_weight': 18}

In [58]:
model=xgb.XGBRegressor(**best_params)

In [59]:
model.fit(x_train,y_train)

XGBRegressor(alpha=0.008738322932750241, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.6662300719496674,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=18, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [60]:
y_pred=model.predict(x_test)

In [61]:
r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.7009858549961854

In [62]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(x_train,y_train)
y_pred2=model2.predict(x_test)
r2_score(y_test,y_pred2)


0.7707509172386131